## Design of Experiments to optimize perovskite solar cells efficiency
Version 1.0.0
(c) Vincent M. Le Corre, Larry Lueer, i-MEET 2021-2023

This notebook is made to use BOAR to design experiments. Here, we show how to load some data from a presampling, and how to use BOAR to suggest the next set of experiment using Bayesian optimization.
The goal here is to optimize the processing conditions for a perovskite solar cell to maximize the power conversion efficiency (PCE).

Note: The data used here is real data generated in the [i-MEET](https://www.i-meet.ww.uni-erlangen.de/) and [HI-ERN](https://www.hi-ern.de/de) labs at the university of Erlangen-Nuremberg (FAU) by Jiyun Zang. The data is not published yet, and is only used here for demonstration purposes. For more information, please contact us.

In [ ]:
# Activate matplotlib widgets
%matplotlib inline
# comment the next line if you are on the jupyterhub server
%matplotlib widget 
# %matplotlib notebook
# import plotly.io as pio # comment out to only render png
# pio.renderers.default = 'png'coconda activate pero

# Import libraries
import sys,os,types,copy
import warnings
import pandas as pd
warnings.filterwarnings('ignore') # comment this out to see warnings

# Import boar
# sys.path.append('/home/vlc/Desktop/boar') # comment out if the Notebook is in the Notebooks folder
from boar import *
# import boar
from boar.core.optimization_botorch import *
# import additional libraries from Ax
from ax.utils.notebook.plotting import render, init_notebook_plotting # for plotting in notebook
from ax.plot.slice import plot_slice
from ax.plot.scatter import interact_fitted,plot_objective_vs_constraints,tile_fitted
from ax.modelbridge.cross_validation import cross_validate
from ax.plot.contour import interact_contour
from ax.plot.diagnostic import interact_cross_validation
from ax.plot.pareto_utils import compute_posterior_pareto_frontier
from ax.plot.pareto_frontier import plot_pareto_frontier
# Import homemade package by VLC
# import boar.SIMsalabim_utils.plot_settings_screen # to set default plot settings



In [ ]:
# Define the path to the data 
curr_dir = os.getcwd() # current directory
# res_dir = os.path.join(os.path.abspath('../'),'temp') # path to the results directory
res_dir = os.path.join(curr_dir ,'temp') # path to the results directory
# data_dir = os.path.join(os.path.abspath('../'),'Example_Data') 
data_dir = ''

In [ ]:
names =['Spin_Speed_1_PbI2', 'Duration_t1', 'Spin_Speed_2_FAI', 'FAI_Dispense_Speed','Duration_t3', 'Spin_Speed_3', 'Pmax']
df = pd.read_excel(os.path.join(data_dir,'BOAR_Exp_2_Round 1.xlsx'),usecols=[0,1,2,3,4,5,11],names=names)

In [ ]:
print(df.keys())

In [ ]:
# names =['No.', 'Spin_Speed_1', 'Duration_t1', 'Spin_Speed_2', 'Dispense_Speed','Duration_t3', 'Spin_Speed_3', 'Pmax']
# df = pd.read_excel(os.path.join(data_dir,'Jiyun','Inital Data Set LHS + Devices Results.xlsx'),'First Round',skiprows=[0,2],usecols=[0,3,4,5,6,7,10,17],names=names)
# df = df.dropna()
# df = df.reset_index(drop=True)

# # print stats on the data
# print(df.describe())



In [ ]:
# names =['No.', 'Spin_Speed_1', 'Duration_t1', 'Spin_Speed_2', 'Dispense_Speed','Duration_t3', 'Spin_Speed_3', 'Pmax']
# df_BO = pd.read_excel(os.path.join(data_dir,'Jiyun','Second Round Parameter Set+ Devices Results.xlsx'),usecols=[0,1,2,3,4,5,6,11],names=names)
# df_BO = df_BO.dropna()
# df_BO = df_BO.reset_index(drop=True)
# df_BO['Pmax'] = abs(df_BO['Pmax'])
# # print stats on the data
# print(df_BO.describe())

# # Concatenate the two dataframes
# df = pd.concat([df,df_BO],axis=0)
# df = df.reset_index(drop=True)

In [ ]:
params_names = names[0:-1]
target_names = [names[-1]]
df_filtered = copy.deepcopy(df[params_names+target_names])
df_filtered = df_filtered.drop_duplicates()
df_filtered = df_filtered.dropna()


dic = {'x':[],'y_0':[],'ydyn_0':1}
for num in range(len(df_filtered)):
    dic['x'].append(df_filtered[params_names].iloc[num].values.tolist())
    dic['y_0'].append(df_filtered[target_names[0]].iloc[num])
    

# save to res_dir
with open(os.path.join(res_dir,'old_XY.json'), 'w') as fp:
    json.dump(dic, fp)

print(dic)

In [ ]:
# import boar.SIMsalabim_utils.plot_settings_screen # to set default plot settings
best_pmax = []
best_random = 0
plt.figure(figsize=(12,10))
for idx,p in enumerate(df['Pmax']):
    if idx == 0:
        best_pmax.append(p)
    else:
        if p > best_pmax[-1]:
            best_pmax.append(p)
            if idx < 30:
                best_random = p
        else:
            best_pmax.append(best_pmax[-1])

df['Best Pmax'] = best_pmax

plt.plot(df['Pmax'],'C0o')
plt.plot(df['Best Pmax'],'C3--')
plt.plot([0,50],[best_random,best_random],'C2-.')

plt.xlabel('Experiment No.')
plt.ylabel('PCE [%]')
plt.show()

In [ ]:
# print(len(dic['y_0']))

# Define the free parameters to be optimized

In [ ]:
params = []

Spin_Speed_1 = Fitparam(name = 'Spin_Speed_1_PbI2', val =  1000, lims = [900, 3000], relRange = 1, range_type = 'lin',
            lim_type = 'absolute',optim_type='lin', display_name = 'Spin Speed 1', unit = 'RPM',val_type='int',rescale=False)
params.append(Spin_Speed_1)
Duration_t1 = Fitparam(name = 'Duration_t1', val =  15, lims = [5, 50], relRange = 1, range_type = 'lin',
            lim_type = 'absolute',optim_type='lin', display_name = 'Duration t1', unit = 's',val_type='int'
                        ,rescale=False)
params.append(Duration_t1)
Spin_Speed_2 = Fitparam(name = 'Spin_Speed_2_FAI', val =  1000, lims = [900, 3000], relRange = 1, range_type = 'lin',
            lim_type = 'absolute',optim_type='lin', display_name = 'Spin Speed 2', unit = 'RPM',val_type='int'
                        ,rescale=False)
params.append(Spin_Speed_2)
Dispense_Speed = Fitparam(name = 'FAI_Dispense_Speed', val =  100, lims = [10, 400], relRange = 1, range_type = 'lin',
            lim_type = 'absolute',optim_type='lin', display_name = 'Dispense Speed', unit = '',val_type='int'
                        ,rescale=False)
params.append(Dispense_Speed)
Duration_t3 = Fitparam(name = 'Duration_t3', val =  10, lims = [5, 45], relRange = 1, range_type = 'lin',
            lim_type = 'absolute',optim_type='lin', display_name = 'Duration t3', unit = 's',val_type='int'
                        ,rescale=False)
params.append(Duration_t3)
Spin_Speed_3 = Fitparam(name = 'Spin_Speed_3', val =  1000, lims = [1000, 5000], relRange = 1, range_type = 'lin',
            lim_type = 'absolute',optim_type='lin', display_name = 'Spin Speed 3', unit = 'RPM',val_type='int',rescale=False)
params.append(Spin_Speed_3)

In [ ]:
# create an excel file with len(params) columns and nb_new_exp rows filled with nan
nb_new_exp = 9
dat_array = np.zeros((nb_new_exp, len(params))).tolist()
# replace all with 'nan'
for i in range(len(dat_array)):
    for j in range(len(dat_array[i])):
        dat_array[i][j] = 'nan'
pnames = [ p.name for p in params ]
df_ = pd.DataFrame(dat_array, columns=pnames)
df_.to_excel(os.path.join(res_dir,'BOAR_Exp.xlsx'), index=False)

df2 = pd.read_excel(os.path.join(res_dir,'BOAR_Exp.xlsx'))
print(df2)
def exp_to_df(X,params):
    pass

# Start the optimization

In [ ]:
# Define the targets and the model for the TM problem
X_dimensions = ['_']
y_dimension = 'Pmax'
target = {'model':exp_to_df,'target_name':'Pmax','minimize':False,
          'data':{'X':[0],'y':[0],'X_dimensions':X_dimensions,'X_units':['V',''],'y_dimension':y_dimension,'y_unit':'mW cm$^{-2}$'}
            ,'target_weight':1, 'weight':1}


targets = [target]
obj_type='identity'
loss='linear'
threshold=[18]

mo = MooBOtorch(params=params, targets= targets) # initialize the optimization object
mo.warmstart = 'recall'
mo.parallel = True # needed if number of cores is lower than number of CPU - 1 to ensure we output the right number of points
# mo.SaveOldXY2file = os.path.join(res_dir,'old_XY.json')
mo.Path2OldXY = os.path.join(res_dir,'old_XY.json')
# mo.parameter_constraints = [f'{stepsize_fraction}*Cs_fraction + {stepsize_fraction}*Fa_fraction <= 1']

# Define custom evaluation function
def evaluate_custom(self,px,obj_type,loss,threshold=1,is_MOO=True):
  pass
  

mo.evaluate_custom = types.MethodType(evaluate_custom, mo) # add the method to the object FullyBayesianMOO
kwargs_posterior = {'Nres':10,'Ninteg':1e3,'logscale':True,'vmin':1e-100,'zoom':0,'min_prob':1e-40,'clear_axis':False,'True_values':None,'show_points':True,'savefig':False,'figname':'param_posterior','full_grid':True,'randomize':True}

ax_client = mo.BoTorchOpti(n_jobs=[nb_new_exp], n_step_points = [nb_new_exp], models=['FullyBayesian'],obj_type=obj_type,loss=loss,threshold=threshold,use_CUDA=True,is_MOO=False,verbose=True,show_posterior=False,kwargs_posterior=kwargs_posterior,use_custom_func=True,suggest_only=True)

In [ ]:
# Print the optimized parameters
for p in mo.params:
    if p.val_type != 'str':
        print(p.display_name + f' {p.val:.0f} ')
        print(p.lims)
    else:
        print(p.display_name + f' {p.val}')

In [ ]:
# get all tried data from the ax_client
triedX = ax_client.generation_strategy.trials_as_df
# print(triedX.tail())
triedY = ax_client.experiment.fetch_data().df
# print(triedY.tail())

# find Trial Status ABANDONED
abandoned_trials = triedX[triedX['Trial Status']=='ABANDONED']
dics = []
for index, row in abandoned_trials.iterrows():
    
    dic_dum = abandoned_trials['Arm Parameterizations'][index]
    key = list(dic_dum.keys())[0]
    dic = dic_dum[key]
    dics.append(dic)

# put in a dataframe
df2try = pd.DataFrame(dics)

# save to excel
df2try.to_excel(os.path.join(res_dir,'BOAR_Exp_2_try.xlsx'), index=False)

    

In [ ]:
# Plot the Pareto front of the test problem
mo.plot_all_objectives(ax_client,logscale=False,figsize=(10,10))

In [ ]:
# Plot the density of points that were sampled during the optimization process
# import boar.SIMsalabim_utils.plot_settings_screen # to set default plot settings
mo.plot_density(ax_client,figsize=(10,10))

In [ ]:
# Plot the contour of the objective function for a given target

render(ax_client.get_contour_plot(param_x=params[0].name, param_y=params[-1].name, metric_name=obj_type))
render(ax_client.get_contour_plot(param_x=params[0].name, param_y=params[1].name, metric_name=obj_type))
render(ax_client.get_contour_plot(param_x=params[0].name, param_y=params[-2].name, metric_name=obj_type))



In [ ]:
# Plot the slice (i.e., 1D projection) of the model along the a single dimension 
model = ax_client.generation_strategy.model

render(plot_slice(model, params[-1].name, obj_type))



In [ ]:
import ax
x = ax.plot.slice.plot_slice(model=model,param_name= params[-1].name, metric_name= obj_type).data['data'][1]['x']

In [ ]:
# Plot the results of cross validation
cv_results = cross_validate(model)
render(interact_cross_validation(cv_results))

In [ ]:
# Interactive plot of the target during the optimization process
render(interact_fitted(model, rel=False))